In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC, SVC
from scipy.stats import uniform, randint
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn import model_selection, linear_model, metrics


In [18]:
df_train = pd.read_csv("../data/train_new.csv", sep="|")
df_val = pd.read_csv("../data/val_new.csv", sep="|")
df_train['totalScannedLineItems'] = df_train['scannedLineItemsPerSecond'] * df_train['totalScanTimeInSeconds']
df_val['totalScannedLineItems'] = df_val['scannedLineItemsPerSecond'] * df_val['totalScanTimeInSeconds']
y_train = df_train.fraud
X_train = df_train.drop(['fraud'], axis=1)
y_val = df_val.fraud
X_val = df_val.drop(['fraud'], axis=1)
df_train.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud,totalScannedLineItems
0,4,828,66.56,7,4,3,0.007246,0.080386,1.166667,0,6.0
1,1,1612,31.34,2,4,3,0.008685,0.019442,0.142857,0,14.0
2,3,848,52.37,2,4,0,0.022406,0.061757,0.105263,0,19.0
3,1,321,76.03,8,7,2,0.071651,0.236854,0.347826,0,23.0
4,1,660,6.06,3,7,1,0.027273,0.009182,0.166667,0,18.0


In [19]:
def display_scores(scores):
    print("Scores: {0}\nMean: {1:.3f}\nStd: {2:.3f}".format(scores, np.mean(scores), np.std(scores)))



def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [22]:
lin_svc = SVC()
params = {
    "kernel": ['linear','poly', 'rbf','sigmoid'],
    "degree": np.arange(1,5),
    "shrinking": [True, False],
    "tol": [1e-4, 1e-3, 5e-4, 5e-3,5e-2, 1e-2]
}

In [ ]:
ftwo_scorer = metrics.make_scorer(metrics.fbeta_score, beta=3)
search = RandomizedSearchCV(lin_svc, scoring=ftwo_scorer, param_distributions=params, random_state=42, n_iter=300,
                            cv=3, verbose=1, n_jobs=-1, return_train_score=True)
search.fit(X_train, y_train)

report_best_scores(search.cv_results_, 3)

c:\users\theur\documents\uni\master\dmc19\venv\lib\site-packages\sklearn\model_selection\_search.py:271: UserWarning: The total space of parameters 192 is smaller than n_iter=300. Running 192 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 192 candidates, totalling 576 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  7.3min
